In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("C:/Users/thinh.lv/google-maps-scraper/output/all/csv/places-of-all.csv")

In [5]:
df_no_duplicates = df.drop_duplicates()

In [6]:
df_no_duplicates

,place_id,name,main_category,phone,address
0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"Joily Coffee, Tea & Bakery, A10 Ng. Đức Hạnh 3..."
1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"Du Miên Garden Cafe, 7 Đ. Phan Văn Trị, Phường..."
13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"Country Houses Coffee, 18 Đ. Phan Văn Trị, Phư..."
23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"Xofa Café & Bistro, 14 P. Tống Duy Tân, Hàng B..."
25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ..."
...,...,...,...,...,...
2413,ChIJ-WnxKyaJoDERR6rIzeW_WdE,Dehavi Coffee,Quán cà phê,NaN,"2P8P+7FM Dehavi Coffee, Nguyễn Văn Cừ Nối Dài,..."
2414,ChIJ5V0eWDdjoDERrTBDf1nIGzY,Take Away Coffee ( HẠ ),Quán cà phê,0942 819 113,"Take Away Coffee ( HẠ ), 46 Đ. Trương Định, Tâ..."
2415,ChIJBf8k86BioDERG2hx89DdiJ8,Cà Phê Minh Thanh,Quán cà phê,0292 3821 147,"Cà Phê Minh Thanh, 91 Đ. Ngô Quyền, Tân An, Ni..."
2416,ChIJaVP7kKBioDERXg7hXg2zXPQ,Cà Phê Mode,Quán cà phê,0292 3819 141,"Cà Phê Mode, 6 Hẻm 22 Đ. Mạc Đĩnh Chi, Tân An,..."


In [7]:
import requests

def get_location(place_id):
    api_keys = ["mwA5L1GvWFvnwIYL-U1kP0RDsZ61ElOKAH7wnbsTs4M",
                "br4m9W3KMhrPa51Vru5M1mWJYCfaVFvWiP3kB2gEQUA",
                "spdVYGb-b6cgokZBOuwGG4b3WbmgsxQqv7tudjdOfLc",
                "0zeR-GEwo5Od-poIZYTGrw-I80fC2C0b0q4F1Boa8oc",
                "UfHD68dH4O9ld-NS22Se6Bls98P9aPZhFq4hA_H7-74",
                "c6NrKs_BpAX4OdRFKAVSDeUaQTjSpNF8dCxjcLT6AfE"]
    current_key_index = 0
    api_key = api_keys[current_key_index]
    url = f'https://revgeocode.search.hereapi.com/v1/revgeocode?apikey={api_key}&id={place_id}'
    
    try:
        # Gửi yêu cầu API
        response = requests.get(url)
        
        # Nếu API key đã hết hạn, thử API key tiếp theo trong danh sách
        key_count = 0
        while response.status_code == 429:
            key_count += 1
            if key_count == len(api_keys):
                return { "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
            current_key_index = current_key_index + 1
            api_key = api_keys[current_key_index]
            url = f'https://revgeocode.search.hereapi.com/v1/revgeocode?apikey={api_key}&at={lat},{long}'
            response = requests.get(url)
            
        # Xử lý kết quả và trả về
        data = response.json()
        address = data['items'][0]['address']
        city = address.get('county', '')
        district = address.get('city', '')
        ward = address.get('district', '')
        street = address.get('street', '')
        houseNumber = address.get('houseNumber', '')
        result = { "House_number": houseNumber, "Street": street, "Ward": ward, "District": district, "City": city, "progress": "True" }
    except:
        result = { "House_number": "", "Street": "", "Ward": "", "District": "", "City": "", "progress": "False" }
    
    return result


In [8]:
# Đọc file đầu vào
df2 = df_no_duplicates
# Lặp qua tất cả address
address_info_list = []
for address in df2["place_id"]:
    address_info = get_location(address)
    address_info_list.append(address_info)

# Tạo ra một DataFrame mới từ list này và thêm vào DataFrame gốc
address_df = pd.DataFrame(address_info_list)
df10 = pd.concat([df2, address_df], axis=1)

In [9]:
df10

,place_id,name,main_category,phone,address,House_number,Street,Ward,District,City,progress
0,ChIJcWJ70U6rNTERxFCKqdDKXrw,"Joily Coffee, Tea & Bakery",Quán cà phê,0703 666 668,"Joily Coffee, Tea & Bakery, A10 Ng. Đức Hạnh 3...",,,,,,False
1,ChIJKbnmWQApdTERwsotmjhioIE,Du Miên Garden Cafe,Quán cà phê,028 3894 5555,"Du Miên Garden Cafe, 7 Đ. Phan Văn Trị, Phường...",,,,,,False
13,ChIJbUggYwcpdTERceZS9v1RSFY,Country Houses Coffee,Quán cà phê,0903 704 399,"Country Houses Coffee, 18 Đ. Phan Văn Trị, Phư...",,,,,,False
23,ChIJ32cXs5erNTERsqdVf0cP-mg,Xofa Café & Bistro,Quán cà phê,024 3717 1555,"Xofa Café & Bistro, 14 P. Tống Duy Tân, Hàng B...",,,,,,False
25,ChIJiXZG2IqrNTERdSRq4nRh9U4,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",Quán cà phê,1800 6503,"Lofita - Chuỗi quán Trà & Cà Phê (Cafe) Ngon, ...",,,,,,False
...,...,...,...,...,...,...,...,...,...,...,...
729,NaN,NaN,NaN,NaN,NaN,,,,,,False
730,NaN,NaN,NaN,NaN,NaN,,,,,,False
732,NaN,NaN,NaN,NaN,NaN,,,,,,False
733,NaN,NaN,NaN,NaN,NaN,,,,,,False
